```{contents}
```
## **Long-Term Memory in LangGraph**

**Long-term memory** in LangGraph refers to the persistent storage of information **across executions, sessions, and time**, enabling agents and workflows to **learn from prior interactions, retain knowledge, and adapt behavior**.
It complements the in-graph **state (short-term memory)** and transforms LangGraph systems from stateless pipelines into **continuous, evolving intelligent systems**.

---

### **1. Motivation and Intuition**

Without long-term memory:

| System Property       | Limitation              |
| --------------------- | ----------------------- |
| Stateless execution   | No learning over time   |
| One-off tasks         | No user personalization |
| No historical context | Repeated mistakes       |

With long-term memory:

| Capability           | Result                   |
| -------------------- | ------------------------ |
| Persistent knowledge | Cumulative learning      |
| Personalization      | User-aware behavior      |
| Experience reuse     | Faster, better decisions |
| Adaptive policies    | Long-lived agents        |

---

### **2. Memory Architecture in LangGraph**

LangGraph distinguishes **three memory layers**:

| Layer                | Scope            | Lifetime |
| -------------------- | ---------------- | -------- |
| **State**            | Single graph run | Short    |
| **Checkpoint**       | Thread / session | Medium   |
| **Long-Term Memory** | Across threads   | Long     |

```
User → Graph Execution → State
               ↓
           Checkpoint Store
               ↓
        Long-Term Memory Store
```

---

### **3. What Is Stored in Long-Term Memory**

| Category           | Examples                            |
| ------------------ | ----------------------------------- |
| User preferences   | writing style, language, interests  |
| Knowledge facts    | domain info, project details        |
| Task history       | previous plans, failures, solutions |
| Policies           | custom rules, constraints           |
| Learned embeddings | semantic memory                     |

---

### **4. Storage Backends**

| Backend                               | Purpose             |
| ------------------------------------- | ------------------- |
| Vector DB (FAISS, Pinecone, Weaviate) | Semantic memory     |
| Relational DB (Postgres)              | Structured memory   |
| Key-Value Store (Redis)               | Fast lookup         |
| Document Store (MongoDB)              | Unstructured memory |
| Object Storage (S3)                   | Large artifacts     |

---

### **5. Memory Ingestion Workflow**

```
Execution → Extract Important Info → Encode → Store → Index
```

#### Example: Storing Knowledge After Task

```python
def store_memory(state):
    memory = state["final_answer"]
    vectorstore.add_texts([memory])
    return {}
```

---

### **6. Memory Retrieval Workflow**

```
New Input → Query Memory → Retrieve → Inject into State → Execute
```

```python
def retrieve_memory(state):
    results = vectorstore.similarity_search(state["input"])
    return {"memory_context": results}
```

---

### **7. Integrating Memory into LangGraph**

```python
class State(TypedDict):
    input: str
    memory_context: list
    output: str
```

```python
builder.add_node("retrieve", retrieve_memory)
builder.add_node("reason", llm_reason)
builder.add_node("store", store_memory)

builder.set_entry_point("retrieve")
builder.add_edge("retrieve", "reason")
builder.add_edge("reason", "store")
builder.add_edge("store", END)
```

---

### **8. Variants of Long-Term Memory**

| Type              | Function            |
| ----------------- | ------------------- |
| Episodic Memory   | Past interactions   |
| Semantic Memory   | Facts & knowledge   |
| Procedural Memory | How-to knowledge    |
| Preference Memory | User choices        |
| Policy Memory     | Constraints & rules |

---

### **9. Production Considerations**

| Concern        | Strategy            |
| -------------- | ------------------- |
| Memory growth  | TTL & summarization |
| Data privacy   | Encryption & RBAC   |
| Relevance      | Periodic pruning    |
| Latency        | Caching layer       |
| Hallucinations | Memory verification |
| Cost           | Tiered storage      |

---

### **10. Safety & Governance**

* Explicit memory write policies
* Human approval for sensitive memory
* Versioned memory entries
* Audit logging
* Compliance controls (GDPR, HIPAA)

---

### **11. Why Long-Term Memory Enables Agents**

Long-term memory converts:

> **Stateless workflows → Persistent intelligent systems**

It enables:

* Continuous learning
* Personalization
* Institutional knowledge
* Autonomous improvement

---

### **12. Mental Model**

Think of LangGraph long-term memory as:

> **The knowledge base of an evolving AI organism**

State = working memory
Checkpoint = short-term recall
Long-term memory = lifetime experience

### Demonstration

In [3]:
from langchain_classic.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(["System initialized"], embeddings)


In [4]:
from typing import TypedDict, List

class State(TypedDict):
    input: str
    memory_context: List[str]
    output: str


In [5]:
def retrieve_memory(state: State):
    results = vectorstore.similarity_search(state["input"], k=2)
    memories = [doc.page_content for doc in results]
    return {"memory_context": memories}


In [15]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

def reason(state: State):
    prompt = f"""
Past knowledge:
{state['memory_context']}

User query:
{state['input']}
"""
    response = llm.invoke(prompt)
    return {"output": response.content}

In [16]:
def store_memory(state: State):
    vectorstore.add_texts([state["output"]])
    return {}


In [17]:
from langgraph.graph import StateGraph, END

builder = StateGraph(State)

builder.add_node("retrieve", retrieve_memory)
builder.add_node("reason", reason)
builder.add_node("store", store_memory)

builder.set_entry_point("retrieve")
builder.add_edge("retrieve", "reason")
builder.add_edge("reason", "store")
builder.add_edge("store", END)

graph = builder.compile()


In [18]:
result = graph.invoke({"input": "Explain reinforcement learning"})
print(result["output"])


Reinforcement learning is a type of machine learning algorithm where an agent learns to make decisions by interacting with an environment and receiving rewards or penalties based on its actions. The goal of reinforcement learning is for the agent to maximize its accumulated rewards over time by learning the optimal policy or strategy to follow in different situations. This involves the agent learning through trial and error, exploring different actions and observing the consequences to improve its decision-making abilities. Reinforcement learning is commonly used in various applications such as autonomous driving, game playing, robotics, and more.
